In [ ]:
import argparse
from enum import Enum
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
import time
import random


In [ ]:
#Instalar gecko
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service


#Abrir un navegador
from selenium import webdriver
from selenium.webdriver.common.by import By
driver = webdriver.Firefox(service=Service(executable_path=GeckoDriverManager().install()))

In [ ]:
URL="https://technical.city/en/video/rating"

In [ ]:

def waitFinishLoad(driver,idLoader,needsDisapeared=False,by=By.ID,needsWait=True):
    """Espera a la carga de un elemento antes de continuar

    Args:
        driver (WebDriver): WebDriver
        idLoader (str): Id a buscar, puede ser XPath...
        needsDisapeared (bool, optional): Si el elemento a esperar tiene que aparecer o desaparecer como un CircularProgress. Defaults to False.
        by (str, optional): Tipo de By a buscar el Wait. Defaults to By.ID.
        needsWait (bool, optional): Si necesita esperar un extra para que no pueda saltar un Captcha. Defaults to False.
    """
    if idLoader is None:
        return
    WebDriverWait(driver, timeout=60).until(EC.presence_of_element_located((by, idLoader)))
    if needsDisapeared:
        WebDriverWait(driver, timeout=60).until(EC.invisibility_of_element((by, idLoader)))
    if needsWait:
        time.sleep(random.choice([0.05,0.1,0.2]))

def elementBy(query,driver,**kargs):
    """Devuelve un element|None dependiendo de la consulta

    Args:
        query (str): Busqueda a realizar del elemento
        driver (WebDriver): WebDriver

    Returns:
        Element|None: Elemento de la búsqueda
    """
    if not kargs.get("needsWait",False):
        waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    element=driver.find_element(kargs.get("by",By.XPATH),query)
    waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    return element

def clickEvent(driver,element,**kargs):
    """Click en un elemento que funciona con evento

    Args:
        driver (WebDriver): WebDriver
        element (Element): Elemento que tiene el evento
    """
    if not kargs.get("needsWait",False):
        waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    if kargs.get("normal",False):
        element.click
    else:
        try:
            driver.execute_script(element.get_dom_attribute("onClick"))
        except:
            element.click
    waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
# endregion

In [ ]:
# id-footer
import pandas as pd

df = pd.DataFrame([])
while True:
    